# Feature Engineering For Ashrae Energy Predcition

### importing libraries

In [ ]:
!pip install meteocalc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from tqdm import tqdm_notebook as tqdm
import datetime
from meteocalc import feels_like, Temp
from sklearn import metrics
import gc
import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
pd.set_option("display.max.rows", None)


In [ ]:
train_df = pd.read_csv('/content/gdrive/My Drive/train.csv')
building_df = pd.read_csv('/content/gdrive/My Drive/building_metadata.csv')
weather_df = pd.read_csv('/content/gdrive/My Drive/weather_train.csv')
weather_test_df = pd.read_csv('/content/gdrive/My Drive/weather_test.csv')
test_df = pd.read_csv('/content/gdrive/My Drive/test.csv')

#### When we are loading the datasets, I dropped floor_count due to the 75% of the data is missing. For the building dataset.

#### tranform the primary_use column to a numeric column by using one of the encoding strategies.

## missing value filling

In [ ]:

def fill_weather_dataset(weather_df):
    
    # Add Day,Week & Month features This dataset consits of hourly weather information. 
    # So we are going to fill missing values based on below new date features.
    # Add new Features
    weather_df["datetime"] = pd.to_datetime(weather_df["timestamp"])
    # adding day ,moth and week as features
    weather_df["day"] = weather_df["datetime"].dt.day 
    weather_df["week"] = weather_df["datetime"].dt.week
    weather_df["month"] = weather_df["datetime"].dt.month
    
    # Reset Index for Fast Update
    weather_df = weather_df.set_index(['site_id','day','month'])
     
    # fill missing air temperature with mean temperature of day of the month. 
    # Each month comes in a season and temperature varies lots in a season. So filling with yearly mean value is not a good idea.
    
    air_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['air_temperature'].mean(),columns=["air_temperature"])
    weather_df.update(air_temperature_filler,overwrite=False)


    #Data is missing for most of days and even many consecutive days. So, first, calculate mean cloud_coverage of day of the month 
    # then fill rest missing values with last valid observation.(fillna with the method='ffill')
    
    cloud_coverage_filler = weather_df.groupby(['site_id','day','month'])['cloud_coverage'].mean()
    cloud_coverage_filler = pd.DataFrame(cloud_coverage_filler.fillna(method='ffill'),columns=["cloud_coverage"]) # imputing with daily means per site id
    weather_df.update(cloud_coverage_filler,overwrite=False)

    #fillna with the method='ffill' option. 'ffill' stands for 'forward fill' and will propagate last valid observation forward
    
    due_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['dew_temperature'].mean(),columns=["dew_temperature"])
    weather_df.update(due_temperature_filler,overwrite=False)  

    
    # Data is missing for most of days and even many consecutive days. So, first, calculate mean sea_level of day of the month 
    # then fill rest missing values with last valid observation.(fillna with the method='ffill')
    sea_level_filler = weather_df.groupby(['site_id','day','month'])['sea_level_pressure'].mean()
    sea_level_filler = pd.DataFrame(sea_level_filler.fillna(method='ffill'),columns=['sea_level_pressure'])
    weather_df.update(sea_level_filler,overwrite=False)

    wind_direction_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_direction'].mean(),columns=['wind_direction'])
    weather_df.update(wind_direction_filler,overwrite=False)

    wind_speed_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_speed'].mean(),columns=['wind_speed'])
    weather_df.update(wind_speed_filler,overwrite=False)


    precip_depth_filler = weather_df.groupby(['site_id','day','month'])['precip_depth_1_hr'].mean()
    precip_depth_filler = pd.DataFrame(precip_depth_filler.fillna(method='ffill'),columns=['precip_depth_1_hr'])
    weather_df.update(precip_depth_filler,overwrite=False)

    weather_df = weather_df.reset_index()
    weather_df = weather_df.drop(['datetime','day','week','month'],axis=1)
  

### meteorological_features

#### Adding relative humidity  value with the help of given dew temperature and ait temperature 

https://bmcnoldy.rsmas.miami.edu/Humidity.html

#### RH: =100*(EXP((17.625*TD)/(243.04+TD))/EXP((17.625*T)/(243.04+T)))

#### adding feel like meteological feature by importing 'meteocalc' library

In [ ]:
  
    def get_meteorological_features(data):
        def calculate_rh(df):
            df['relative_humidity'] = 100 * (np.exp((17.625 * df['dew_temperature']) / (243.04 + df['dew_temperature'])) / np.exp((17.625 * df['air_temperature'])/(243.04 + df['air_temperature'])))
        def calculate_fl(df):
            flike_final = []
            flike = []
            # calculate Feels Like temperature
            for i in range(len(df)):
                at = df['air_temperature'][i]
                rh = df['relative_humidity'][i]
                ws = df['wind_speed'][i]
                flike.append(feels_like(Temp(at, unit = 'C'), rh, ws))
            for i in range(len(flike)):
                flike_final.append(flike[i].f)
            df['feels_like'] = flike_final
            del flike_final, flike, at, rh, ws
        calculate_rh(data)
        calculate_fl(data)
        return data

    weather_df = get_meteorological_features(weather_df)
    
    return weather_df


In [ ]:
def features_engineering(df):
    
    # Sort by timestamp
    df.sort_values("timestamp")
    df.reset_index(drop=True)
    
    # Add more features
    df["timestamp"] = pd.to_datetime(df["timestamp"],format="%Y-%m-%d %H:%M:%S")
    df["hour"] = df["timestamp"].dt.hour
    df["dayofweek"] = df["timestamp"].dt.weekday

    df['group'] = df['timestamp'].dt.month
    df['group'].replace((1, 2, 3, 4), 1, inplace = True)
    df['group'].replace((5, 6, 7, 8), 2, inplace = True)
    df['group'].replace((9, 10, 11, 12), 3, inplace = True)
   
    return df

In [ ]:
weather_df = fill_weather_dataset(weather_df)
weather_test_df = fill_weather_dataset(weather_test_df)

In [ ]:
weather_df  = features_engineering(weather_df )
weather__test_df  = features_engineering(weather_test_df )
train_df  = features_engineering(train_df)
test_df  = features_engineering(test_df)

### Adding Holiday features for each site id

In [ ]:
location=pd.DataFrame()
location['site_id']=np.arange(0,16)

##https://www.kaggle.com/datadugong/locate-better-cities-by-weather-temp-fill-nans
##https://www.kaggle.com/c/ashrae-energy-prediction/discussion/115698
##https://www.kaggle.com/patrick0302/locate-cities-according-weather-temperature

location['city']=['Orlando','Heathrow','Tempe','Washington','Berkeley','Southampton',\
                     'Washington','Ottowa','Orlando','Austin','Saltlake','Ottowa','Dublin',\
                      'Minneapolis','Philadelphia','Rochester']
location['country']=['US','UK','US','US','US','UK',\
                    'US','Montreal','US','US','US','Montreal','Ireland',\
                    'US','US','US']
weather_df= weather_df.merge(location, on='site_id', how='left')
weather_test_df= weather_test_df.merge(location, on='site_id', how='left')

In [ ]:
from datetime import date
## https://www.geeksforgeeks.org/python-holidays-library/
## https://towardsdatascience.com/5-minute-guide-to-detecting-holidays-in-python-c270f8479387
import holidays
UK=[]
for ptr in holidays.UnitedKingdom(years=2016).keys():
    UK.append(str(ptr))
for ptr in holidays.UnitedKingdom(years=2017).keys(): #2017 year holydays in uk
    UK.append(str(ptr))
for ptr in holidays.UnitedKingdom(years=2018).keys():
    UK.append(str(ptr))
    UK.append('2019-01-01')
IR=[]
for ptr in holidays.Ireland(years=2016).keys():  #2016 year holydays in ireland
    IR.append(str(ptr))
for ptr in holidays.Ireland(years=2017).keys():
    IR.append(str(ptr))
for ptr in holidays.Ireland(years=2018).keys():
    IR.append(str(ptr))
    IR.append('2019-01-01')
US=[]
for ptr in holidays.UnitedStates(years=2016).keys(): #2016 year holydays in US
    US.append(str(ptr))
for ptr in holidays.UnitedStates(years=2017).keys():
    US.append(str(ptr))
for ptr in holidays.UnitedStates(years=2018).keys():
    US.append(str(ptr))
    US.append('2019-01-01')
CA=[]
for ptr in holidays.Canada(years=2016).keys():   #2016 year holydays in Canada
    CA.append(str(ptr))
for ptr in holidays.Canada(years=2017).keys():
    CA.append(str(ptr))
for ptr in holidays.Canada(years=2018).keys():
    CA.append(str(ptr))
    CA.append('2019-01-01')

In [ ]:
def is_holiday(df):
    df['is_holiday']=[0]*(df.shape[0])
    df.loc[df['country']=='US','is_holiday']=(df['timestamp'].dt.date.astype('str').isin(US)).astype(int)
    df.loc[df['country']=='UK','is_holiday']=(df['timestamp'].dt.date.astype('str').isin(UK)).astype(int)
    df.loc[df['country']=='Montreal','is_holiday']=(df['timestamp'].dt.date.astype('str').isin(CA)).astype(int)
    df.loc[df['country']=='Ireland','is_holiday']=(df['timestamp'].dt.date.astype('str').isin(IR)).astype(int)
    return df
# get the holiday
weather_df=is_holiday(weather_df)
weather_test_df=is_holiday(weather_test_df)

In [ ]:


train_df = train_df.merge(building_df, left_on='building_id',right_on='building_id',how='left')
train_df = train_df.merge(weather_df,how='left',left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])

test_df = test_df.merge(building_df,left_on='building_id',right_on='building_id',how='left')
del building_df
gc.collect()



test_df = test_df.merge(weather_test_df,how='left',on=['timestamp','site_id'])
del weather_df
gc.collect()


# transform target variable
train_df['meter_reading'] = np.log1p(train_df["meter_reading"])
train_df['square_feet'] =  np.log1p(train_df['square_feet'])

test_df['square_feet'] =  np.log1p(test_df['square_feet'])
    
    # Remove Unused Columns
     # Encode Categorical Data
le = LabelEncoder()
train_df["primary_use"] = le.fit_transform(train_df["primary_use"])
test_df["primary_use"] = le.fit_transform(test_df["primary_use"])

## removing outliers and floor_count feature

In [ ]:
train_df.drop('floor_count',axis=1,inplace=True)
test_df.drop('floor_count',axis=1,inplace=True)

#### As we saw in Eda building_id 1099 is considered as outier 

#### and we are also removing rows with electricity =0 

In [ ]:
idx_to_drop = list(train_df[(train_df['meter'] == 0) & (train_df['meter_reading'] == 0)].index)
train_df.drop(idx_to_drop,axis='rows',inplace=True)

In [ ]:
idx_to_drop = list(train_df[(train_df['building_id'] == 1099)].index)
train_df.drop(idx_to_drop,axis='rows',inplace=True)

In [ ]:
train_df = train_df.drop(['timestamp','country','city','hour_y','dayofweek_y','group_y','hour_x'], axis = 1)
test_df = test_df.drop(['timestamp','country','city','hour_y','dayofweek_y','group_y','hour_x'], axis = 1)

In [ ]:
test_df.to_csv('/content/gdrive/My Drive/test_preprocessed1.csv')

## summery of Notebook 

1) Filling missed values:

    *  As cloud_coverage, precip_depth_1_hr, sea_level_pressure and wind_direction have significantly high missing values need to fill it

    * Add Day,Week & Month features This dataset consits of hourly     weather information. So we are going to fill missing values based on  below new date features.
  
    a) fill missing air temperature with mean temperature of day of the month. Each month comes in a season and temperature varies lots in a season. So filling with yearly mean value is not a good idea.

    b) Data is missing for most of days and even many consecutive days. So, first, calculate mean cloud_coverage, precip_depth_1_hr, sea_level_pressure and wind_direction  of day of the month and then fill rest missing values with last valid observation.(fillna with the method='ffill' option. 'ffill' stands for 'forward fill' and will propagate last valid observation forward)

2) Adding Features to data:

   a)meteorological_features:
    
    * as dew_temperatur and air_temperature were given 
    
    * Relative_humidity : =100(EXP((17.625 * dewtemperature)/(243.04+dew_temperature))/EXP((17.625*air_temperature)/(243.04+air_temperature)))
    
    * By using this relation  added relative humidity feature to dataset 

    * feel_like feature: 
    
    * We calculate a 'feels like temperature by taking into account the expected air temperature, relative humidity and the strength of the wind 
    
    * As we have all features to find feel like temperature i used meteocalc library (https://pypi.org/project/meteocalc/)
    to get it done
    

    b) Holiday_features
    
    * we got assumed contries for each site_id while  digging each  variable in data

    * https://www.kaggle.com/datadugong/locate-better-cities-by-weather-temp-fill-nans
    
    * https://www.kaggle.com/c/ashrae-energy-prediction/discussion/115698
    
    * https://www.kaggle.com/patrick0302/locate-cities-according-weather-temperature
    
    * by using python in build holiday library (contains holydays for most of the contries) we get holidays for us,uk,canada        and irland .
    
    * using these added the is_holiday feature to the dataset (https://towardsdatascience.com/5-minute-guide-to-detecting-holidays-in-python-c270f8479387)

    * Then encoded the catogorical features 

3) Removal of Outliers

     * As we found in eda building 1099 is an outlier we removed it

     * There is no reason keeping zero electrical readings as it cant be zero .so those rows are also excluded from dataset 
     
     * floor_coont feature is removed as it contains more than 76 percent of misssing values    